# import 

In [1]:
import cv2

In [2]:
import random

In [3]:
import os

In [24]:
from fastprogress.fastprogress import progress_bar

In [25]:
import numpy as np

# functions

In [5]:
# 关注的水体的灰度值。
#0 未标注。 1 空地。 2 建筑 。3 水体。4 道路
TYP_WATER = 3

In [6]:
#水体占比在这个值一下的认为是空的
BLANK_TH = 0.04

In [35]:
def gen_dataset(imgfns, maskfns, output_dir = 'label/dataset_20200708', imgdir = 'image'
                , maskdir = 'mask', img_sz = (512, 512), ds_sz = 1000, blank_pct = 0.2
                , suffix = '.png', dbg = []):
    '''
    生成数据集
    参数：
        imgfns：大图片文件名列表
        maskfns：对应的mask文件名列表。与上一个参数里面的文件一一对应
        output_dir：输出数据集目录
        imgdir：输出数据集中存放图片的子目录。
        maskdir：输出数据集中存放mask图片的子目录
        img_sz：提取的小图的大小,内容为(width, height)
        ds_sz：生成图片数量
        blank_pct：无水体图片数量比例
        suffix：生成图片文件扩展名
        dbg：调试用
    '''
    assert len(imgfns) == len(maskfns)
    #检查创建目录
    for sdir in ['', imgdir, maskdir]:
        if not os.path.exists(os.path.join(output_dir, sdir)):
            os.makedirs(os.path.join(output_dir, sdir))
    total_cnt = 0#产出计数
    imgs = []
    for imgfn, maskfn in zip(imgfns, maskfns):
        img = cv2.imread(imgfn)
        mask = (cv2.imread(maskfn, cv2.IMREAD_GRAYSCALE) == TYP_WATER).astype(int) * 255
        #print(imgfn, maskfn, img.shape, mask.shape)
        assert img.shape[ : 2] == mask.shape[ : 2]
        imgs += [(img, mask)]
    
    #当前大图需要产生的空白数据图片的数量
    DS_SZ_BLANK = int(ds_sz * blank_pct)        
    #需要产生的有效图片的数量
    DS_SZ_VALID = ds_sz - DS_SZ_BLANK

    validcnt = 0 #当前图片产生的数量
    blankcnt = 0#当前产出的空白数据集的数量
    #print('dbg1', DS_SZ_1IMG, DS_SZ_1IMG_BLANK, DS_SZ_1IMG_VALID)
    
    #随机生成一个问题是分类区域占比太少的情况下会导致前期生成的大部分都是空白图片。    #
    gen_idxs = [i for i in range(ds_sz)]    
    random.shuffle(gen_idxs)
    print('gen_idxs', gen_idxs)
    
    pbar = progress_bar(range(ds_sz))
    pbar.comment = '生成中'
    for p in pbar:
        while validcnt < DS_SZ_VALID or blankcnt < DS_SZ_BLANK:
            idx = random.randint(0, 2)
            #print(idx, imgfns[idx])
            img, mask = imgs[idx]
            dsx = random.randint(0, img.shape[0] - img_sz[1])
            dsy = random.randint(0, img.shape[1] - img_sz[0])
            dsimg = img[dsx : dsx + img_sz[1], dsy : dsy + img_sz[0]]            
            dsmask = mask[dsx : dsx + img_sz[1], dsy : dsy + img_sz[0]]        
            area = cv2.countNonZero(dsmask)
            #本次想要生产的类别.用数值来确定。
            cat = random.randint(0, ds_sz - 1)            
            #cat = np.random.randint(0, ds_sz)
            
            #根据比例判断一下是否有效的
            if area / (img_sz[0] * img_sz[1]) > BLANK_TH:
                #print(validcnt, total_cnt, ds_sz % len(imgfns))
                #这次准备生成的是空白的，得到的是有内容的
                #if cat < DS_SZ_BLANK:
                #    continue
                #超过了数量的放弃.并且保证到达整数
                if validcnt >= DS_SZ_VALID:
                    continue
                validcnt += 1
            else:
                #这次应该是有效内容，但是得到的是空白的
                #if cat >= DS_SZ_BLANK:
                #    continue
                if blankcnt >= DS_SZ_BLANK:
                    continue
                blankcnt += 1

            cv2.imwrite(os.path.join(output_dir, imgdir, '%05d.%s' % (gen_idxs[total_cnt], suffix.split('.')[-1])), dsimg)
            cv2.imwrite(os.path.join(output_dir, maskdir, '%05d_mask.%s' % (gen_idxs[total_cnt], suffix.split('.')[-1])), dsmask)
            #cv2.imwrite(os.path.join(output_dir, imgdir, '%05d.%s' % (total_cnt, suffix.split('.')[-1])), dsimg)
            #cv2.imwrite(os.path.join(output_dir, maskdir, '%05d_mask.%s' % (total_cnt, suffix.split('.')[-1])), dsmask)

            total_cnt += 1
            break


# test

In [36]:
imgfns = ['data/src/image/1.png', 'data/src/image/2.png', 'data/src/image/3.png']

In [37]:
maskimgfns = ['data/label/train1_labels_8bits.png', 'data/label/train2_labels_8bits.png'
             , 'data/label/train3_labels_8bits.png']

In [38]:
gen_dataset(imgfns, maskimgfns, ds_sz = 200, suffix='jpg', output_dir = 'label/dataset_20200713') 

gen_idxs [134, 13, 23, 33, 74, 163, 2, 106, 12, 142, 150, 76, 90, 87, 116, 197, 41, 31, 20, 65, 54, 44, 78, 192, 79, 149, 35, 24, 194, 135, 151, 175, 124, 105, 147, 185, 188, 17, 176, 34, 80, 164, 25, 160, 120, 26, 152, 184, 103, 187, 18, 100, 183, 84, 177, 61, 69, 28, 36, 72, 104, 11, 157, 62, 15, 97, 7, 143, 144, 117, 158, 118, 1, 75, 71, 82, 4, 48, 77, 83, 95, 115, 180, 171, 46, 190, 30, 133, 148, 167, 57, 159, 70, 191, 42, 88, 138, 19, 108, 153, 14, 114, 47, 131, 137, 22, 21, 0, 123, 66, 186, 140, 27, 98, 92, 37, 67, 96, 132, 174, 162, 101, 94, 169, 113, 166, 173, 58, 121, 29, 129, 119, 198, 40, 81, 141, 93, 170, 155, 107, 85, 45, 68, 199, 139, 181, 59, 126, 99, 112, 89, 51, 189, 122, 168, 55, 156, 154, 5, 3, 136, 125, 110, 32, 52, 63, 50, 127, 8, 60, 16, 73, 111, 196, 109, 195, 128, 49, 6, 193, 165, 86, 130, 178, 161, 53, 102, 39, 91, 146, 9, 179, 172, 43, 182, 64, 10, 38, 56, 145]


# export